In [ ]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH
from syft_rds.syft_runtime.main import CodeRuntime

In [ ]:
stack = setup_rds_stack(log_level="INFO")

In [ ]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a custom runtime as DO

In [ ]:
CWD = RDS_NOTEBOOKS_PATH / "quickstart"

In [ ]:
private_dir = CWD / "data" / "custom-runtime-1" / "private"
mock_dir = CWD / "data" / "custom-runtime-1" / "mock"
markdown_path = CWD / "data" / "custom-runtime-1" / "description.md"

private_dir.mkdir(parents=True, exist_ok=True)
mock_dir.mkdir(parents=True, exist_ok=True)


private_script_path = private_dir / "script.py"
mock_script_path = mock_dir / "script.py"

In [ ]:
%%writefile {private_script_path}

import os

INPUT_FILE = os.environ["INPUT_FILE"] # user input file
OUTPUT_DIR = os.environ["OUTPUT_DIR"] # job output directory


SECRET_KEY = os.environ["SECRET_KEY"] # secret key for some api

def special_string_length(query: str):
    return len(query + SECRET_KEY)

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    res = special_string_length("Hello, world!")
    f.write(f"Query result: {res}")
    print(f"Query result: {res}")


In [ ]:
# python do_owner.py ds_code.jpg

In [ ]:
%%writefile {mock_script_path}

import os

INPUT_FILE = os.environ["INPUT_FILE"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]
SECRET_KEY = "MOCK_KEY"

def special_string_length(query: str):
    return len(query + SECRET_KEY)

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    res = special_string_length("Hello, world!")
    f.write(f"Query result: {res}")
    print(f"Query result: {res}")


In [ ]:
with open(markdown_path, "w") as f:
    f.write("You input should be file with a single string")

In [ ]:
data = do_client.dataset.create(
    name="custom_runtime_dataset",
    path=private_dir,
    mock_path=mock_dir,
    summary="dummy data",
    description_path=markdown_path,
    runtime=CodeRuntime(
        cmd=[
            "python",
            private_script_path.as_posix(),
        ],  # set the entrypoint for the runner
        mount_dir=private_dir.as_posix(),  # make the folder available to the container
    ),
)

In [ ]:
# docker python -c my/local/path

# Use dataset as DS

In [ ]:
ds_client.datasets

In [ ]:
ds_client.dataset.get(name="custom_runtime_dataset").get_mock_path()

In [ ]:
dataset = ds_client.dataset.get(name="custom_runtime_dataset")
dataset.describe()

In [ ]:
my_file = CWD / "data" / "input.txt"

In [ ]:
%%writefile {my_file}

My special string


In [ ]:
job = ds_client.jobs.submit(user_code_path=my_file, dataset_name=dataset.name)
job

In [ ]:
job = do_client.jobs.get_all()[-1]
job

In [ ]:
import os

os.environ["SECRET_KEY"] = "__AA__"  # Secret used in the private code
res_job = do_client.run_private(job)

In [ ]:
job = ds_client.jobs.get_all()[-1]

In [ ]:
job._syftbox_client

In [ ]:
ds_client._syftbox_client.datasites

In [ ]:
job.output_url

In [ ]:
job.get_output_path()